# Mail Model per hour

## Importing mail datas

In [ ]:
import awswrangler as wr
import os
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scikit-learn.model_selection import cross_val_score
from scikit-learn.metrics import make_scorer
from scikit-learn.metrics import mean_absolute_error as mae
from scikit-learn.metrics import mean_absolute_percentage_error as mape

from gives import *

def get_db_from_prefix(db_prefix: str) -> str:
    """
    Get the name of the database with the shared suffix if it exists otherwise
    return the input db_prefix.
    You can get the db_prefix in the overview tab of the dataset in DataStudio
    from the "Glue database" field in the "AWS information" section.
    """
    shared_suffixed_name = f'{db_prefix}_shared_'
    database_names = wr.catalog.databases()["Database"]
    return next(filter(lambda x: x.startswith(shared_suffixed_name), database_names), db_prefix)

In [ ]:
databases = wr.catalog.databases()
db_prefix = 'im_commercial_excellence_sales_eu_shared_oeflowob' 
database = get_db_from_prefix(db_prefix)
tables = wr.catalog.tables(database=database, limit=50)
table = tables["Table"].iloc[5]
table

In [ ]:
df_mail = wr.athena.read_sql_query(
    sql=f'''SELECT case_number, consideration_start, reason
            FROM "{table}"
            WHERE to_email_address = 'contact.alfi@airliquide.com'
              AND consideration_start >= TIMESTAMP '2020-05-01 00:00:00' ''',
    database=database,
    ctas_approach=False,
    workgroup=os.getenv("ATHENA_WORKGROUP")
)#.sort_valuers('consideration_start',ascending=False)

df_mail.fillna('vide',inplace=True)
df_mail

The **f** before the string in the command **f'SELECT case_number, consideration_start, reason FROM "{table}" WHERE to_email_address = \'contact.alfi@airliquide.com\' AND consideration_start >= TIMESTAMP \'2020-06-01 00:00:00\''** is used to create a formatted string literal, also known as an f-string in Python.

The **f** prefix allows you to include expressions inside curly braces **{}** within the string, which will be evaluated and replaced with their values. In this case, the **{table}** is a placeholder for the value of the variable table, which will be substituted with the actual table name when the string is formatted.

The backslash **\\** is used as an escape character in this context. It is used to escape the single quotes (**'**) within the string literal. By adding a backslash before the single quote (**\'**), it indicates that the single quote is part of the string and should not be interpreted as the end of the string.

For example, in the string **\'contact.alfi@airliquide.com\'**, the backslashes before the single quotes ensure that the single quotes are treated as literal characters within the string, rather than as string delimiters.

This way, the **f** prefix and the backslash escape **\\** characters help in constructing the SQL query as a valid string with the necessary substitutions and escaping of special characters.

## Exploring data: cleaning and analyzing

### Cleaning

In [ ]:
df_mail.reason.unique()
#df_mail.loc[df_mail['reason']=='Payment']

In [ ]:
#reason_table = df_mail.pivot_table(values='case_number', columns='reason', aggfunc='count')
px.histogram(df_mail,x='reason').update_xaxes(categoryorder='total descending')

In [ ]:
df_mail_2023=df_mail.set_index('consideration_start').sort_index().loc['2023':,:]
px.histogram(df_mail_2023, x='reason').update_xaxes(categoryorder='total descending')

In [ ]:
mapp_link=pd.read_csv('../data/mapping_\'reason\'_mail.csv')
print(mapp_link.to_dict())
dict_link={'logistic': ['Logistics and material orders', 'Distribution/logistic', 'Liquid ordering & Delivery', 
                        'C4 - Liquid ordering & Delivery', 'C3 - Cylinder Ordering & Delivery', 'Cylinder ordering & Delivery', 
                        'C5 - Equipement & Installations'],
 'ecopass': 'Ecopass',
 'distributor': 'Support to distributors',
 'contract': ['Contract & Customer Relationship', 'Customer Digital Platform Support', 'Copy of document', 
              'Administration & Master Data', 'Maintenance', 'Product Quality', 'Cylinders packaging and maintenance', 'Quality', 
              'Maintenance & Services', 'C2 - Contract & Cust. Relationship', 'Installations Hard goods', 'Transactional Survey', 
              'Administrative'],
 'invoicing': ['Invoicing & Payment', 'Payment', 'C7 - Invoicing & Payment', 'C8 - Payment', 'Cash Collection'],
 'others': ['vide', 'Others', 'Other informations', 'Mistake / Silence / Joke', 'Covid-19', 'Other Group Company', 'Suggestions', 
            'Relationship Survey', 'S9 - IT/IS Systems', 'S8 - Cylinders Package & Maintenance', 'C6 - Maintenance & Services', 
            'S5 - Product Quality']}

In [ ]:
def reas_cl(value) :
    if value in ['Logistics and material orders', 'Distribution/logistic', 'Liquid ordering & Delivery', 
                 'C4 - Liquid ordering & Delivery', 'C3 - Cylinder Ordering & Delivery', 'Cylinder ordering & Delivery', 
                 'C5 - Equipement & Installations'] :
        value = 'logistic'
    elif value in ['Ecopass'] :
        value = 'ecopass'
    elif value in ['Support to distributors'] :
        value = 'distributor'
    elif value in ['Contract & Customer Relationship', 'Customer Digital Platform Support', 'Copy of document', 
                   'Administration & Master Data', 'Maintenance', 'Product Quality', 'Cylinders packaging and maintenance', 'Quality', 
                   'Maintenance & Services', 'C2 - Contract & Cust. Relationship', 'Installations Hard goods', 'Transactional Survey', 
                   'Administrative','Commercial'] :
        value = 'contract'
    elif value in ['Invoicing & Payment', 'Payment', 'C7 - Invoicing & Payment', 'C8 - Payment', 'Cash Collection'] :
        value = 'invoicing'
    elif value in ['vide', 'Others', 'Other informations', 'Mistake / Silence / Joke', 'Covid-19', 'Other Group Company', 'Suggestions', 
                   'Relationship Survey', 'S9 - IT/IS Systems', 'S8 - Cylinders Package & Maintenance', 'C6 - Maintenance & Services', 
                   'S5 - Product Quality', 'S1 - Risks & Environment'] :
        value = 'others'
    return value

def in_list(value, list_) :
    if value in list_ :
        value = True
    else :
        value = False
    return value

In [ ]:
df_mail_clean = df_mail.copy()
df_mail_clean.reason = df_mail.reason.apply(reas_cl)
df_mail_clean.reason.unique()

### Analysing statistics and distributions

In [ ]:
fig=px.histogram(df_mail_clean, x='reason').update_xaxes(categoryorder='total descending')
fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="orange",
    title_font_size=20,
    #legend_title_font_color="green",
    
)
fig.update_layout(
    title={
        'text': "Répartition des mails par équipe",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

In [ ]:
df_clean_2023=df_mail_clean.set_index('consideration_start').sort_index().loc['2023':,:]
fig=px.histogram(df_clean_2023, x='reason',labels={"reason": "Equipe", "count" : "Volumes"}).update_xaxes(categoryorder='total descending')
fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="orange",
    title_font_size=20,
    #legend_title_font_color="green",
    
)
fig.update_layout(
    title={
        'text': "Répartition des mails par équipe en 2023",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

'Others' will be considered as an epsilon error. Two ways of modelating it : one with an lgbm for every categories including one for others or one regression model where others would be considered as an epsilon error as it was said earlier.

#### Zoom sur 'others'

In [ ]:
df_others=df_mail.loc[df_mail.reason.apply(in_list, list_=['vide', 'Others', 'Other informations', 'Mistake / Silence / Joke', 'Covid-19', 'Other Group Company', 'Suggestions', 
                   'Relationship Survey', 'S9 - IT/IS Systems', 'S8 - Cylinders Package & Maintenance', 'C6 - Maintenance & Services', 
                   'S5 - Product Quality', 'S1 - Risks & Environment']),:]#.set_index('consideration_start').sort_index().loc['2023':,:]
fig=px.histogram(df_others, x='reason',labels={"reason": "Equipe", "count" : "Volumes"}).update_xaxes(categoryorder='total descending')
fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="orange",
    title_font_size=20,
    #legend_title_font_color="green",
    
)
fig.update_layout(
    title={
        'text': "Zoom sur 'others'",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

In [ ]:
df_others=df_mail.loc[df_mail.reason.apply(in_list, list_=['vide', 'Others', 'Other informations', 'Mistake / Silence / Joke', 'Covid-19', 'Other Group Company', 'Suggestions', 
                   'Relationship Survey', 'S9 - IT/IS Systems', 'S8 - Cylinders Package & Maintenance', 'C6 - Maintenance & Services', 
                   'S5 - Product Quality', 'S1 - Risks & Environment']),:].set_index('consideration_start').sort_index().loc['2023':,:]
fig=px.histogram(df_others, x='reason',labels={"reason": "Equipe", "count" : "Volumes"}).update_xaxes(categoryorder='total descending')
fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="orange",
    title_font_size=20,
    #legend_title_font_color="green",
    
)
fig.update_layout(
    title={
        'text': "Zoom sur 'others' en 2023",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

### Forming dataframes

In [ ]:
dico={}
for name in df_mail_clean.reason.unique() :
    dico["df_" + str(name)] = df_mail_clean.loc[df_mail_clean.reason==name]
dico['df_invoicing'].head(7)

In [ ]:
#par heure 
# d_new={}
# for i in dico : 
#     d_new[i] = df_agg(dico[i],freq='H',y_name='volume', key='consideration_start')

In [ ]:
#par jour
d_new={}
for i in dico : 
    key='consideration_start'
    d_new[i] = dico[i].groupby(pd.Grouper(key='consideration_start', freq='D')).size().reset_index(name='volume')
    # Ajout des features temporelles détaillées
    d_new[i]=d_new[i].assign(year=d_new[i][key].dt.isocalendar()['year'], 
                 week=d_new[i][key].dt.isocalendar()['week'],
                 day=d_new[i][key].dt.isocalendar()['day'],
                 #quarter=d_new[i]['Consideration Start'].dt.quarter,
                 month=d_new[i][key].dt.month,
                 day_mon=d_new[i][key].dt.day,)
                 #hour=d_new[i][key].dt.hour*100+d_new[i][key].dt.minute)

    # Conversion de la colonne de date en index
    d_new[i].set_index(key, inplace=True)

    # Éjection des samedis et dimanches lorsque l'on est en phone only
    d_new[i].drop(index=d_new[i].loc[(d_new[i]['day']==6) | (d_new[i]['day']==7)].index, inplace=True)

    # Conversion dans le type adéquat pour LGBM
    d_new[i] = d_new[i].astype(int)

    # Rajoute les vacances, jours fériés et d-1 fériés
    d_new[i]=d_new[i].assign(vac_A=d_new[i].apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'A'),axis=1),
                 vac_B=d_new[i].apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'B'),axis=1),
                 vac_C=d_new[i].apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'C'),axis=1),
                 ouvre=d_new[i].apply(lambda x : cal.is_working_day(datetime.date(x['year'],x['month'],x['day_mon'])),axis=1),
                 # begin=d_new[i].apply(lambda x : 800<=x['hour']<900, axis=1),
                 # lunch=d_new[i].apply(lambda x : 1200<=x['hour']<1300, axis=1),
                 # end=d_new[i].apply(lambda x : 1700<=x['hour']<1800, axis=1)
                 )


    d_new[i]['d-1_fer']=((d_new[i]['ouvre'].shift(1,fill_value=True)-1)*-1).apply(bool)
    d_new[i].loc[(d_new[i]['day']==1) & (d_new[i]['d-1_fer'])==True,'d-1_fer']=False 

    # Éjection des samedis et dimanches lorsque l'on est en phone only ainsi que les jours fériés
    d_new[i].drop(index=d_new[i].loc[d_new[i].ouvre==False].index, inplace=True)
    d_new[i].drop(columns='ouvre', inplace=True)

In [ ]:
df=d_new['df_invoicing']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Vraies valeurs", x=df.index, y=df.volume, mode='lines+markers'))

## Models' Training

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df, train_beg='2020-05-01', train_end='2022-05-31', test_beg='2022-06-01', test_end='2023-06-09', y_name='volume')

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    #'cat_feature=name' : ['year', 'week', 'day','quarter', 'month', 'day_mon', 'vac_A', 'vac_B', 'vac_C', 'lunch', 'd-1_fer']
}

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(model)

### Hyperopt

#### LGBM_hopt_1

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

list_obj = ['tweedie','mae','regression']

lgb_reg_params = { ##définition perso des valeurs possibles
    'objective' :       'regression',
    #'metric':           hp.choice('metric', list_obj), #‘perso_pic’
    'learning_rate':    hp.uniform('learning_rate',0.001,0.5), # ou ,0.1,1)
    #'num_iterations':   hp.choice('num_iterations',       np.arange(5, 200, 1, dtype=int)),
    'num_leaves':       hp.choice('num_leaves',       np.arange(2, 89, 1, dtype=int)),
    'max_depth':        hp.choice('max_depth',        np.arange(2, 100, 1, dtype=int)),
    #'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
    #'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
    #'subsample':        hp.uniform('subsample', 0.6, 1),
    #'min_split_gain':   hp.uniform('min_split_gain', 0, 1),
}
# lgb_reg_params_comp = {
#     'objective' :       hp.choice('objective', ['tweedie','mae','regression']), #‘perso_pic’
#     'learning_rate':    hp.uniform('learning_rate',0.1,1),
#     'num_leaves':       hp.choice('num_leaves',       np.arange(2, 200, 1, dtype=int)),
#     'max_depth':        hp.choice('max_depth',        np.arange(2, 100, 1, dtype=int)),
#     'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
#     'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
#     'subsample':        hp.uniform('subsample', 0.6, 1),
#     'min_split_gain':   hp.uniform('min_split_gain', 0, 1),
#     'n_estimators':     5
# } 
##### mettre les espaces des paramètres compris dans Param_LGBM
def f(params) : 
    lgbm = lgb.LGBMRegressor(n_jobs=-1,early_stopping_rounds=None,**params)
    score = cross_val_score(lgbm, X_train, y_train, cv=2,scoring=mae_scorer,n_jobs=-1).mean() ## ['regression', 'tweedie', 'mape'] je peux les mettre ici sinon en utilant scikit-learn...cross_validate
    return score

In [ ]:
trials = Trials()
result = fmin(
    fn=f,                           # objective function
    space=lgb_reg_params,   # parameter space
    algo=tpe.suggest,               # surrogate algorithm
    max_evals=500,                  # no. of evaluations
    trials=trials,                   # trials object that keeps track of the sample results (optional)
    verbose=1
)
result
result = space_eval(lgb_reg_params, result)

In [ ]:
result

In [ ]:
params = result.copy()
#params['objective']='tweedie'
params['num_iterations']=200

lgb_hopt = lgb.LGBMRegressor(**params)
lgb_hopt.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(lgb_hopt)

#### LBGM_hopt_2

In [ ]:
params = {'learning_rate': 0.1630936299000586,
 'max_depth': 60,
 'num_leaves': 2,
 'objective': 'regression'}

# {'learning_rate': 0.010059706014775412,
#  'max_depth': 83,
#  'num_iterations': 200,
#  'num_leaves': 82,
#  'objective': 'tweedie'}

# {'colsample_bytree': 0.41495667971584227,
#  'learning_rate': 0.31545383409933103,
#  'max_depth': 66,
#  'min_child_weight': 16,
#  'min_split_gain': 0.17155337882245103,
#  'num_leaves': 48,
#  'objective': 'regression',
#  'subsample': 0.9107569191768009}

lgb_hopt_2 = lgb.LGBMRegressor(**params)
lgb_hopt_2.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(lgb_hopt_2)

In [ ]:
# train_lgb = lgb.Dataset(data = X_train, label = y_train, feature_name = list(X_train))
# cv_results = lgb.cv(params,train_lgb,stratified=False, shuffle=False, num_boost_round=1)

### Test Plot

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name="Vraies valeurs", x=y_test.index, y=y_test, mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=y_test.index, y=model.predict(X_test).astype(int), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_HP", x=y_test.index, y=lgb_hopt.predict(X_test).astype(int), mode='lines+markers'))
#fig.add_trace(go.Scatter(name="LGBM_HP_2", x=y_test.index, y=lgb_hopt_2.predict(X_test).astype(int), mode='lines+markers'))


mean=(model.predict(X_test)+lgb_hopt_2.predict(X_test))/2
#fig.add_trace(go.Scatter(name="LGBM_HP_mean", x=y_test.index, y=mean.astype(int), mode='lines+markers'))

fig.show()

#### Plot Aver. Perc. Err.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="LGBM", x=X_test.index, y=predict_test(model, pd.DataFrame(), X_test, y_test)["absolute_error_%"], mode='lines+markers'))

#### Learning & Test Error

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Vraies valeurs", x=pd.concat([X_train, X_test]).index, y=pd.concat([y_train, y_test]), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=X_train.index, y=model.predict(X_train), mode='lines+markers', marker_color='springgreen'))
fig.add_trace(go.Scatter(name="LGBM_hopt", x=X_train.index, y=lgb_hopt.predict(X_train), mode='lines+markers', marker_color='indianred'))

fig.add_trace(go.Scatter(name="LGBM_test", x=X_test.index, y=model.predict(X_test).astype(int), mode='lines+markers', marker_color='#00CC96'))
fig.add_trace(go.Scatter(name="LGBM_hopt_test", x=X_test.index, y=lgb_hopt.predict(X_test).astype(int), mode='lines+markers',  marker_color='#EF553B'))
fig.show()

In [ ]:
tabl_err_all([model, lgb_hopt, lgb_hopt_2], X_train, y_train, X_test, y_test)

In [ ]:
tabl_err_all([model, lgb_hopt, lgb_hopt_2], X_train, y_train, X_test.loc['2023-01-01':'2023-04-01'], y_test.loc['2023-01-01':'2023-04-01']) 

In [ ]:
import joblib
# save model
joblib.dump(lgb_hopt, 'lgbm_reg_D_mail_invoicing.pkl')
# load model
gbm_pickle = joblib.load('lgbm_reg_D_mail_invoicing.pkl')